## PARAMS & overview
Ho centralizzato i path e definito gli output principali per preparare i file destinati a Tableau.
- Uso questo notebook per: caricare il file cleaned, applicare la rimozione outlier sul prezzo (IQR), creare il file riga-per-riga per la mappa e l'aggregato per dealer_region.
- Modifica i path soltanto in questa cella se lavori su un altro computer.

In [9]:
# PARAMS: definisco qui i path canonici (modificare solo se necessario)
import os
import pandas as pd
import numpy as np

# nota: assumo che il notebook sia in project_root/notebook/
ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))  # se il notebook è altrove, aggiusta qui
RAW_DIR = os.path.join(ROOT, "data", "raw")
PROCESSED_DIR = os.path.join(ROOT, "data", "processed")
MAPPINGS_DIR = os.path.join(ROOT, "notebook", "mappings")

# file principali: uso il cleaned canonical fornito da Matteo (o presente in data/processed)
CLEANED_PATH = os.path.join(PROCESSED_DIR, "database_cleaned.csv")
OUT_AGG_PATH = os.path.join(PROCESSED_DIR, "agg_by_dealer_region_for_tableau.csv")
OUT_CITY_PATH = os.path.join(PROCESSED_DIR, "database_for_tableau_city_state.csv")

# controllo rapido — se il file cleaned non esiste prendo nota per ripristino
print("ROOT:", ROOT)
print("CLEANED_PATH exists?", os.path.exists(CLEANED_PATH))
print("PROCESSED_DIR:", PROCESSED_DIR)
print("MAPPINGS_DIR:", MAPPINGS_DIR)

ROOT: /Users/serenatempesta/Documents/Progetti/Data_Analysis/progetto_finale
CLEANED_PATH exists? True
PROCESSED_DIR: /Users/serenatempesta/Documents/Progetti/Data_Analysis/progetto_finale/data/processed
MAPPINGS_DIR: /Users/serenatempesta/Documents/Progetti/Data_Analysis/progetto_finale/notebook/mappings


In [10]:
# Carico il cleaned file in modo robusto (se esiste la colonna Date la converto)
# Se CLEANED_PATH non è presente, intervenire prima di eseguire le celle successive.
cols0 = pd.read_csv(CLEANED_PATH, nrows=0).columns.tolist()
parse_arg = ['Date'] if 'Date' in cols0 else None

df = pd.read_csv(CLEANED_PATH, parse_dates=parse_arg, low_memory=False)
print("Caricato df shape:", df.shape)
print("Colonne presenti:", df.columns.tolist())
display(df.head(3))

Caricato df shape: (23906, 13)
Colonne presenti: ['Date', 'Customer Name', 'Gender', 'Annual Income', 'Dealer_Name', 'Company', 'Model', 'Engine', 'Transmission', 'Color', 'Price ($)', 'Body Style', 'Dealer_Region']


,Date,Customer Name,Gender,Annual Income,Dealer_Name,Company,Model,Engine,Transmission,Color,Price ($),Body Style,Dealer_Region
0,2022-01-02,Geraldine,Male,13500,Buddy Storbeck's Diesel Service Inc,Ford,Expedition,DoubleÂ Overhead Camshaft,Auto,Black,26000,SUV,Middletown
1,2022-01-02,Gia,Male,1480000,C & M Motors Inc,Dodge,Durango,DoubleÂ Overhead Camshaft,Auto,Black,19000,SUV,Aurora
2,2022-01-02,Gianna,Male,1035000,Capitol KIA,Cadillac,Eldorado,Overhead Camshaft,Manual,Red,31500,Passenger,Greenville


In [11]:
## Pulizia minima e regola outlier
- Trasformo Price e Annual Income in numerici (rimuovo simboli).
- Applico outlier detection **solo su Price** usando la regola IQR (1.5 * IQR).
- Tengo due dataframe:
  - `df` = originale (con outlier) — per audit;
  - `df_no_outliers` = senza outlier sul prezzo — usato per salvare i file per Tableau.

SyntaxError: invalid character '—' (U+2014) (3785279869.py, line 5)

In [ ]:
# Pulisco le colonne money e le converto in numerico (mantengo la colonna originale per controllo)
def to_numeric_money(series):
    # rimuovo simboli non numerici, gestisco valori vuoti
    s = series.astype(str).str.replace(r"[^\d\.\-]", "", regex=True)
    s = s.replace("", np.nan)
    return pd.to_numeric(s, errors='coerce')

# riconosco i nomi tipici delle colonne (tolleranza a varianti)
price_cols = [c for c in df.columns if c.lower().strip() in ("price ($)","price","price_$","price($)","price ($)")]
income_cols = [c for c in df.columns if c.lower().strip() in ("annual income","annual_income","income","annualincome")]

if not price_cols:
    raise RuntimeError("Non trovo la colonna Price: controlla i nomi delle colonne nel dataset.")
PRICE_COL = price_cols[0]
INCOME_COL = income_cols[0] if income_cols else None

# creo colonne pulite
# Ho chiamato le colonne _price_clean e _income_clean per non sovrascrivere i dati originali
df["_price_clean"] = to_numeric_money(df[PRICE_COL])
if INCOME_COL:
    df["_income_clean"] = to_numeric_money(df[INCOME_COL])
else:
    df["_income_clean"] = np.nan

print("Pulizia price: valori nulli ->", df["_price_clean"].isna().sum(), 
      "| min/max ->", df["_price_clean"].min(), df["_price_clean"].max())

# Outlier detection SOLO su price (IQR)
q1 = df["_price_clean"].quantile(0.25)
q3 = df["_price_clean"].quantile(0.75)
iqr = q3 - q1
lower = q1 - 1.5 * iqr
upper = q3 + 1.5 * iqr

# segnalo gli outlier in una colonna booleana; li escludo solo per i file di visualizzazione
df["_price_outlier"] = (df["_price_clean"] < lower) | (df["_price_clean"] > upper)
n_outliers = int(df["_price_outlier"].sum())
print(f"Ho identificato {n_outliers} outlier sul prezzo (IQR).")

# creo df_no_outliers usato per le esportazioni verso Tableau
df_no_outliers = df[~df["_price_outlier"]].copy()
print("Shape df_no_outliers:", df_no_outliers.shape)